# NLP with Bert for Sentiment Analysis

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 87 kB/s 
     |████████████████████████████████| 6.8 MB 35.1 MB/s 
     |████████████████████████████████| 981 kB 37.7 MB/s 
     |████████████████████████████████| 263 kB 42.5 MB/s 
     |████████████████████████████████| 1.9 MB 44.9 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s 
     |████████████████████████████████| 468 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=b2d0a0000ced872c80c6843c62a30da5a980f554b20050d4bd066d041420329c
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=37d0e82afe12fe2e8c591377724f114858b6b7625447068348eb6715d22af396
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [2]:
import os.path
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text


الان نقوم بتحميل قاعدة البيانات , حيث نستخدم 

keras 

 لقراءة الملف  

 aclImdb_v1.tar.gz  

والتأكيد علي عمل 

extract 

  له , وسيقوم كولاب بتحميل الملف الي الذاكرة المؤقتة له




In [3]:
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract=True)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

84140032/84125825 [==============================] - 2s 0us/step


و يمكن استعراض الملفات داخلها هكذا 

In [10]:
os.listdir(IMDB_DATADIR)

['README', 'imdbEr.txt', 'train', 'test', 'imdb.vocab']

In [11]:
os.listdir(os.path.join(IMDB_DATADIR , 'train'))

['unsup',
 'urls_pos.txt',
 'urls_unsup.txt',
 'labeledBow.feat',
 'unsupBow.feat',
 'neg',
 'urls_neg.txt',
 'pos']

In [12]:
os.listdir(os.path.join(IMDB_DATADIR , 'train','pos'))[:10]

['2334_9.txt',
 '151_10.txt',
 '11661_7.txt',
 '2511_10.txt',
 '11807_7.txt',
 '7539_10.txt',
 '7727_9.txt',
 '705_10.txt',
 '6509_9.txt',
 '3872_9.txt']

In [4]:
print(os.path.dirname(dataset))
print(IMDB_DATADIR)

/root/.keras/datasets
/root/.keras/datasets/aclImdb


كما يمكن قراءة احد الملفات هكذا

In [13]:
f= open(os.path.join(IMDB_DATADIR , 'train','pos','5035_7.txt'),'r')
for a in f:
    print(a)

McConaughey in a horror/thriller? I had to see this. I was pleasantly surprised.<br /><br />The plot is told in flashback mode, and it concerns an otherwise normal and happy family of three going through a very bizarre predicament. I can't say much more without spoiling the whole movie, sorry. Just know that if you decide to watch it, you'll be, in the very least, surprised.<br /><br />All the main players are very good. Bill Paxton did a great job directing those kids, and his acting is awesome. McConaughey's acting is solid throughout and fits the bill perfectly.<br /><br />This movie challenges you to think. Is Dad crazy? Is there a God? Do Demons exist? How far would you go to right a wrong. And what is "right" anyway? I'm still thinking.<br /><br />And thus I recommend "Frailty". 7/10 and this is one of those movies that deserves and rewards a second, or even third viewing.


الان نقوم بقراءة الداتا و تقسيمها , و ذلك عبر استخدام مكتبة كيترين   و التي ستقرأ الملفات من المسار الموضح,  ثم يتم كتابة اسماء النوعين "ايجابي" , "سلبي", ثم تحديد ان العدد الاقصي للكلمات المستخدمة هو 500 , وتحديد اسم التدريب و الاختبار, واخيرا تحديد نوع المعالجة التي ستتناول بها الكلمات وهو بيرت

In [5]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir=IMDB_DATADIR,
                                                                       classes=['pos','neg'],
                                                                       maxlen=500,
                                                                       train_test_names=['train','test'],
                                                                       preprocess_mode='bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


الان لخطوة بناء الموديل , و التي تتم عبر  كتابة الكود  : 

In [6]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


  
فنقوم بتحديد اسم الموديل المختار , ثم بيانات التدريب , و اخيرا المعالج المستخدم  , والذي سيكون نفسه بريبروك  و الذي خرج من الكود السابق 


الان في مرحلة ضبط خصائص الـ ليرنز  و هي اشبه بخطوة كومبايل في الشبكات العصبية, حيث نقوم بضبط الخصائص قبل التدريب

In [7]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

مع العلم ان الرقم الباتش لابد ان تتناسب عكسيا مع ال ماكس لين  فكلما زادت عدد كلمات كل جملة  ,يجب ان تقل الباتش  و هنا القيم المسموح بها : 

sequence len=64, max_batch_size=64

sequence len=128, max_batch_size=32

sequence len=256, max_batch_size=16

sequence len=320, max_batch_size=14

sequence len=384, max_batch_size=12

sequence len=512, max_batch_size=6

الان خطوة التدريب  نستخدم معها معدل تعلم صغير , ويكفي ايبوك واحد لانه سيكون بطئ جدا و خلال هذا الايبوك ستزداد الدقة بشكل كبير

In [ ]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.2479 - accuracy: 0.8976

ثم خطوة التوقع , والتي ستكون باستخدام دالة التوقع

In [ ]:
y_pred = learner.predict(val_data=(x_test[:100],y_test[:100]))

وقتها تكون القيمة هكذا

In [ ]:
y_pred[:10]